# Segmentação Semântica - Rocks
## Configuração

Importando módulos necessários.

In [ ]:
import os
import time
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from skimage import io
from sklearn import metrics
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Rodando em:', device)

### Definição de funções auxiliares

In [ ]:
def initialize_weights(*models):
    for model in models:
        for module in model.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                if module.bias is not None:
                    module.bias.data.zero_()
                    
            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.fill_(1)
                module.bias.data.zero_()


def evaluate(preds, labels):
    f1_list = []
    iou_list = []

    for i in tqdm(range(len(preds)), desc='Metrics'):
        f1 = metrics.f1_score(labels[i].flatten(), preds[i].flatten())
        iou = metrics.jaccard_score(labels[i].flatten(), preds[i].flatten())

        f1_list.append(f1)
        iou_list.append(iou)

    f1_list = np.asarray(f1_list)
    iou_list = np.asarray(iou_list)

    return f1_list, iou_list

# Introdução

Neste notebook faremos alguns experimentos com o Dataset [DRP-Benchmarks](https://www.sciencedirect.com/science/article/pii/S0098300412003147) sendo utilizados as bases de Bereau e Grosmont. Em seguida treinaremos uma UNet para realizar a segmentação semântica das rochas nas imagens extraidas.

# Dataloaders Customizados

Os frameworks de Deep Learning modernos (i.e. MXNet e Pytorch) permitem a criação de dataloaders customizados ao se sobrescreverem classes desses frameworks. Esse tipo de dataloader é especialmente útil no caso de tarefas diferentes das de classificação que temos visto até agora (i.e. segmentação e detecção de imagens, processamento de áudio, processamento de linguagem natural, etc), nas quais os labels podem ser mais esparsos ou densos que rótulos de classificação.

Usando como base as classes [*Dataloader*](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) e [*Dataset*](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) do subpacote [*data*](https://pytorch.org/docs/stable/data.html) do Pytorch, podemos customizar a leitura dos dados ao mesmo tempo em que paralelizamos a leitura das amostras dos nossos batches. A paralelização da leitura de amostras em várias [threads](https://www.tutorialspoint.com/python/python_multithreading) torna o uso da GPU o mais eficiente possível, já que não é necessário deixar a GPU esperando pelo carregamento de novas amostras para compor um batch.

In [ ]:
class RockDataset(Dataset):
    def __init__(
            self,
            is_train: bool,
            crop_size: int = 256,
            num_classes: int = 2,
            datapath: str = '/pgeoprj2/ciag2024/dados/drp-benchmarks/images/grosmont'
        ):
        
        self.is_train = is_train
        self.crop_size = crop_size
        self.num_classes = num_classes

        self.img_path = os.path.join(datapath, 'tif')
        self.mask_path = os.path.join(datapath, 'segmented-kongju.raw')

        self.image_filenames, self.mask_vol = self.make_dataset()

        if len(self.image_filenames) == 0:
            raise RuntimeError('Found 0 images, please check the data set.')

    def make_dataset(self):
        if self.is_train:
            mask_vol = np.fromfile(open(self.mask_path, 'rb'), dtype=np.int8).reshape(1024, 1024, 1024)
            mask_vol = mask_vol.transpose((0, 2, 1))[:-64]
        else:
            mask_vol = np.fromfile(open(self.mask_path, 'rb'), dtype=np.int8).reshape(1024, 1024, 1024)
            mask_vol = mask_vol.transpose((0, 2, 1))[-64:]

        if self.is_train:
            image_filenames = sorted([f for f in os.listdir(self.img_path) if os.path.isfile(os.path.join(self.img_path, f))])[:-64]
        else:
            image_filenames = sorted([f for f in os.listdir(self.img_path) if os.path.isfile(os.path.join(self.img_path, f))])[-64:]

        return image_filenames, mask_vol

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        filename = self.image_filenames[idx]
        img_path = os.path.join(self.img_path, filename)

        # Lendo os dados
        img = io.imread(img_path)
        mask = self.mask_vol[idx]

        # Fazendo o casting apropriado
        img = img.astype(np.float32)
        mask = mask.astype(np.int64)

        # Z-Score Normalization
        img = (img - img.mean()) / img.std()

        # Realizando crops (treino = random, teste = fixo)
        if self.is_train:
            randh, randw = np.random.randint(0, 1024 - self.crop_size, size=2)

            img = img[randh:randh+self.crop_size, randw:randw+self.crop_size]
            mask = mask[randh:randh+self.crop_size, randw:randw+self.crop_size]

            # Adicionando dimensão para canal
            img = np.expand_dims(img, axis=0)  # (H, W) -> (1, H, W)
            
        else:
            img = np.array([
                img[0:256, 0:256],    img[256:512, 0:256],    img[512:768, 0:256],    img[768:1024, 0:256],
                img[0:256, 256:512],  img[256:512, 256:512],  img[512:768, 256:512],  img[768:1024, 256:512],
                img[0:256, 512:768],  img[256:512, 512:768],  img[512:768, 512:768],  img[768:1024, 512:768],
                img[0:256, 768:1024], img[256:512, 768:1024], img[512:768, 768:1024], img[768:1024, 768:1024],
            ])

            mask = np.array([
                mask[0:256, 0:256],    mask[256:512, 0:256],    mask[512:768, 0:256],    mask[768:1024, 0:256],
                mask[0:256, 256:512],  mask[256:512, 256:512],  mask[512:768, 256:512],  mask[768:1024, 256:512],
                mask[0:256, 512:768],  mask[256:512, 512:768],  mask[512:768, 512:768],  mask[768:1024, 512:768],
                mask[0:256, 768:1024], mask[256:512, 768:1024], mask[512:768, 768:1024], mask[768:1024, 768:1024],
            ])

            # Adicionando dimensão para canal
            img = np.expand_dims(img, axis=1)  # (16, H, W) -> (16, 1, H, W)

            img = np.array(img, dtype=np.float32)
            mask = np.array(mask, dtype=np.int64)
        
        img = torch.from_numpy(img)
        mask = torch.from_numpy(mask)

        return img, mask

In [ ]:
# Configurando datasets e dataloaders
batch_size = 16
num_workers = 2

train_dataset = RockDataset(is_train=True)
valid_dataset = RockDataset(is_train=False)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=None, num_workers=num_workers, shuffle=False)

print('Número de instâncias de treino:', len(train_dataset))
print('Número de instâncias de validação:', len(valid_dataset))

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(16, 8))
for i, (img, label) in enumerate(train_dataset):
    if i >= 4:
        break

    ax[0][i].imshow(img[0].numpy(), cmap=plt.get_cmap('gray'))
    # ax[0][i].imshow(lab.numpy(), 'Greens_r', interpolation='nearest', alpha=0.3)  # Descomente para ver o overlay das labels
    ax[0][i].set_yticks([])
    ax[0][i].set_xticks([])

    ax[1][i].imshow(label.numpy(), cmap=plt.get_cmap('gray'))
    ax[1][i].set_yticks([])
    ax[1][i].set_xticks([])

plt.show()

# Atividade Prática: Implementando pipeline de segmentação

O dataset contém apenas duas classes e um canal de input, tendo os dataloaders já pré-definidos acima. Para essa tarefa vamos usar uma arquitetura `UNet`.

Os passos a serem seguidos são os seguintes:
1.   Definir arquitetura de uma rede de segmentação.
2.   Definir uma loss.
3.   Instanciar um otimizador.
4.   Implementar funções de treino e teste.

Primeiramente vamos implementar a arquitetura da UNet. Para isso, vamos dividir a arquitetura em blocos _encoder_ e blocos _decoder_. Os __encoders_ são usados na parte inicial da rede, e os _decoders_ na parte final.

Cada bloco _encoder_ possui duas camadas de convolução com `kernel_size=3` e `padding=1`, de forma com que essas convoluções não alterem as dimensões espaciais (altura $H$ e largura $W$) da entrada. Apenas no final de cada bloco _encoder_ temos um `max_pooling` para reduzir as dimensões espaciais da entrada (ela terá metade da altura e largura).

Cada bloco _decoder_ possui duas camadas de convolução, da mesma forma dos _encoders_ (sem alterar dimensões espaciais). No final do bloco, para aumentar a dimensão espacial da entrada, ele possui uma convolução transposta com `kernel_size=2` e `stride=2` para dobrar a altura e largura da entrada.

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super(EncoderBlock, self).__init__()

        # [TODO] Cada bloco do encoder será composto por:
        #   1) Conv2d com in_channels e out_channels como entrada e saída;
        #   2) Batch norm;
        #   3) ReLU;
        #   4) Conv2d replicando out_channels como entrada e saída;
        #   5) Batch norm;
        #   6) ReLU;
        #   7) MaxPooling2d com kernel 2x2 e stride de 2.
        
        self.encode = nn.Sequential(
            # implemente sua solução aqui
        )
    
    # TODO: implemente o forward
    def forward(self, x):
        pass


class DecoderBlock(nn.Module):
    def __init__(self, in_channels: int, middle_channels: int, out_channels: int):
        super(DecoderBlock, self).__init__()

        # [TODO] cada bloco do decoder será composto por:
        #   1) Dropout de canais;
        #   2) Conv2d tendo in_channels e middle_channels como entrada e saída;
        #   3) Batch Norm;
        #   4) Conv2d replicando middle_channels como entrada e saída;
        #   5) Batch Norm;
        #   6) ReLU;
        #   7) Conv2d transposta, tendo como saída out_channels com kernel 2x2 e stride de 2.

        self.decode = nn.Sequential(
            # implemente sua solução aqui
        )
    
    # TODO: implemente o forward
    def forward(self, x):
        pass

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels: int, num_classes: int = 2):
        super(UNet, self).__init__()

        # [TODO] Reutilize os blocos encoder e decoder para construir a UNet.
        #        Experimente diferentes formas de organização desses blocos (ex.: com e sem dropout entre eles).
        #        Obs.: Não se esqueça da última camada de classificação (faça ela convolucional)

        # implemente sua solução aqui
        
        # Inicialização dos pesos
        initialize_weights(self)

    def forward(self, x):
        # TODO: implemente o forward nos blocos de encoder.
        
        # implemente sua solução aqui

        # TODO: implemente o forward dos blocos de decoder.
        #       Lembre-se da característica fundamental da UNet nesse passo!

        # implemente sua solução aqui

        # TODO: implemente o forward na última camada (predição de classes)

        # implemente sua solução aqui
        
        return outputs

In [ ]:
model = UNet(in_channels=1)
model = model.to(device)

print(model)

In [ ]:
# Configurando o otimizador
weight_decay = 5e-4
learning_rate = 1e-4

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
# TODO: escolha qual a loss mais apropriada para esse caso
criterion = ...

In [ ]:
def train(model, train_dataloader, criterion, optimizer):
    tic = time.time()

    train_losses = []
    all_labels, all_preds = [], []

    model.train()
    for i, batch in (pbar := tqdm(enumerate(train_dataloader), total=len(train_dataloader), unit='batch')):
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        # TODO: implemente um step do treinamento dessa rede
        # Use outputs e loss para salvar os outputs do modelo e loss
        outputs = ...
        loss = ...

        # Obtendo as predições (argmax)
        preds = outputs.argmax(dim=1)

        # Salvando as predições e labels para computar métricas para toda época.
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        # Atualizando o display da loss
        train_losses.append(loss.item())
        pbar.set_description(f"Train loss: {np.mean(train_losses):.4f}")

    # Computando métricas da época
    f1, iou = evaluate(all_preds, all_labels)
    tac = time.time()

    print('[train], [loss %.4f +/- %.4f], [iou %.4f +/- %.4f], [f1 %.4f +/- %.4f], [time %.2f]' % (
        np.mean(train_losses), np.std(train_losses), iou.mean(), iou.std(), f1.mean(), f1.std(), (tac - tic)))


def validate(model, valid_dataloader, criterion, epoch):
    tic = time.time()
    display_images_freq = 1

    valid_losses = []
    all_labels, all_preds = [], []

    model.eval()
    with torch.no_grad():
        for i, batch in (pbar := tqdm(enumerate(valid_dataloader), total=len(valid_dataloader), unit='batch')):
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            # TODO: implemente um step de validação.
            # Use outputs e loss para salvar a saída da rede e loss

            outputs = ...
            loss = ...

            # Obtendo as predições (argmax)
            preds = outputs.argmax(dim=1)

            # Salvando as predições e labels para computar métricas para toda época.
            all_preds.append(preds.detach().cpu().numpy())
            all_labels.append(labels.detach().cpu().numpy())

            # Atualizando o display da loss
            valid_losses.append(loss.item())
            pbar.set_description(f"Valid loss: {np.mean(valid_losses):.4f}")

            # Exibindo alguns exemplos
            if i == 0 and epoch % display_images_freq == 0:
                fig, ax = plt.subplots(2, 3, figsize=(10, 7))
                perm = np.random.permutation(inputs.size(0))

                for p in range(2):
                    ax[p, 0].imshow(inputs[perm[p], 0].detach().cpu().numpy())
                    ax[p, 0].set_yticks([])
                    ax[p, 0].set_xticks([])
                    ax[p, 0].set_title('Image')

                    ax[p, 1].imshow(labels[perm[p]].detach().cpu().numpy())
                    ax[p, 1].set_yticks([])
                    ax[p, 1].set_xticks([])
                    ax[p, 1].set_title('Label')

                    ax[p, 2].imshow(preds[perm[p]].detach().cpu().numpy())
                    ax[p, 2].set_yticks([])
                    ax[p, 2].set_xticks([])
                    ax[p, 2].set_title('Prediction')

                plt.show()

    # Computando métricas da época
    f1, iou = evaluate(all_preds, all_labels)
    tac = time.time()

    print('[test], [loss %.4f +/- %.4f], [iou %.4f +/- %.4f], [f1 %.4f +/- %.4f], [time %.2f]' % (
        np.mean(valid_losses), np.std(valid_losses), iou.mean(), iou.std(), f1.mean(), f1.std(), (tac - tic)))

In [ ]:
num_epochs = 3

for epoch in range(1, num_epochs + 1):
    print(f' ========== Epoch {epoch} ========== ')

    train(model, train_dataloader, criterion, optimizer)
    validate(model, valid_dataloader, criterion, epoch)